In [1]:
import sys
import os 
module_path = os.path.abspath(os.path.join('..')) 
if module_path not in sys.path: 
    sys.path.append(module_path)

import pandas as pd  
import numpy as np  

import tools.Sample_Tools as smpl
import tools.Pretreat_Tools as pretreat
from tools.Cacher import (CACHE_TYPE, save_cache,load_cache_adv,load_cache)

# import ind.alpha191 as a191


import matplotlib.pyplot as plt
import seaborn as sns

from base.JuUnits import parallal_task,task_chunk_split
from base.JuUnits import excute_for_multidates



# from QUANTAXIS.QAUtil import DATABASE
# from QUANTAXIS.QAUtil import  trade_date_sse
# from QUANTAXIS.QAUtil.QADate_trade import (
#     QA_util_get_pre_trade_date,
#     QA_util_get_next_trade_date,
#     QA_util_if_tradetime
# )

# import inspect

# from sklearn import linear_model

%load_ext autoreload
%autoreload 2
%aimport ind.alpha191

In [2]:
###加载数据

ids = np.arange(1,192)
# np.random.shuffle(ids)
files = list(map(lambda x:'191_alpha'+'0'*(3- int(np.log10(x)+1))+str(x)+'_train',ids))

def load_alpha_factors(file_names):
    from tools.Cacher import (CACHE_TYPE, save_cache,load_cache_adv,load_cache)
    from base.JuUnits import excute_for_multidates
    import numpy as np
    import pandas as pd 
    
    temp = []
    for file in file_names:
        print(file)
        temp.append(load_cache(file, cache_type=CACHE_TYPE.FACTOR))
    
    return pd.concat(temp,axis=1)
        
worker=6
task = task_chunk_split(files, worker)
results = parallal_task(worker, load_alpha_factors, task)
res = pd.concat(results,axis=1)

Now in the main code. Process name is: base.JuUnits
base.JuUnits, subpid:21644  pid:25024


  0%|          | 0/6 [00:00<?, ?it/s]

In [3]:
def filter_invalid_factor(fators_df):
    ## 缺失30%以上，且3倍方差以上，废弃
    ## 加极端判断是为了保证留下那种大窗口期的因子。
    def nan_count(x):
        return round(x.isna().sum(axis=0)/len(x),4)*100
    
    #计算缺失率
    valid_rate = excute_for_multidates(res, nan_count, level=1) 
    valid_distribute_boundary = valid_rate.mean(axis=0) + valid_rate.std(axis=0)*3
    #缺失率符合当前因子在所有标的上，符合3个标准差的
    filter_contition = valid_rate > valid_distribute_boundary 
    
    def filter_invalid(x):
        code = x.index.get_level_values(1)[0]
        distribute_condition = filter_contition.loc[code] 
        #符合3个标准差的列
        invalid_factors_name = distribute_condition[distribute_condition==True].index 
        #符合3个标准差，并且缺失30%以上的列
        thredhold_condition = valid_rate.loc[code][invalid_factors_name]>30 
        
        x[thredhold_condition.index] = np.nan 
        return x

    filtered = excute_for_multidates(fators_df, filter_invalid, level=1)
    
    return filtered

factors_filtered = filter_invalid_factor(res)

In [4]:
def filter_correlative_factor(fators_df, thredhold=.70, show_chart=False,verbose=False):
    #去掉相关性强的因子，相关性大于thredhold的因子（列）将被去除
    corr = fators_df.corr()
    matrix = corr.values
    #取得下三角矩阵
    matrix[np.triu_indices(matrix.shape[0])]=np.nan
    corr_triu = pd.DataFrame(matrix, columns=corr.columns, index=corr.index)

    if show_chart:
        plt.figure(figsize=(30,30))
        labels = corr.columns.str.replace('191_alpha','')
        ax = sns.heatmap(corr_triu, xticklabels=labels, yticklabels=labels, cmap="YlGnBu",cbar=True)

    # 取得符合条件的因子对后，再取“一对的左边”作为删除目标
    correlative_columns_name = corr_triu.abs().gt(thredhold).stack().loc[lambda x: x].index.get_level_values(0).unique().to_list()
    
    if verbose:
        print('delete correlative factors: {}'.format(','.join(correlative_columns_name)))
        
    correlative_droed = fators_df.drop(columns=correlative_columns_name)
    return correlative_droed

factors_filtered = filter_correlative_factor(factors_filtered,show_chart=False,verbose=True)

delete correlative factors: 191_alpha008,191_alpha014,191_alpha020,191_alpha022,191_alpha024,191_alpha026,191_alpha027,191_alpha028,191_alpha031,191_alpha034,191_alpha037,191_alpha040,191_alpha046,191_alpha047,191_alpha049,191_alpha050,191_alpha051,191_alpha052,191_alpha053,191_alpha055,191_alpha057,191_alpha058,191_alpha059,191_alpha062,191_alpha065,191_alpha066,191_alpha067,191_alpha068,191_alpha071,191_alpha072,191_alpha078,191_alpha079,191_alpha082,191_alpha088,191_alpha089,191_alpha090,191_alpha094,191_alpha095,191_alpha096,191_alpha097,191_alpha099,191_alpha100,191_alpha105,191_alpha106,191_alpha112,191_alpha116,191_alpha121,191_alpha122,191_alpha126,191_alpha128,191_alpha129,191_alpha132,191_alpha133,191_alpha134,191_alpha136,191_alpha137,191_alpha139,191_alpha143,191_alpha147,191_alpha150,191_alpha153,191_alpha154,191_alpha156,191_alpha160,191_alpha161,191_alpha162,191_alpha165,191_alpha167,191_alpha168,191_alpha173,191_alpha174,191_alpha175,191_alpha177,191_alpha180,191_alpha1

In [26]:
# with pd.option_context("display.max_rows", None, "display.max_columns", None):

# codes = factors_filtered.index.get_level_values(1).unique()
# dates = factors_filtered.index.get_level_values(0).unique()
# data_replenish = smpl.get_data(codes.to_list(),dates.min().strftime('%Y-%m-%d'),dates.max().strftime('%Y-%m-%d')).data
# mv_industry = smpl.add_marketvalue_industry(data_replenish).drop(columns=data_replenish.columns.to_list())
# industy = mv_industry['industry']
# cap = mv_industry['totalCapital']

# adj_weight = np.sqrt(cap)/np.sqrt(cap).sum()


# adj_weight_mat = pd.DataFrame(np.diag(adj_weight),index=adj_weight.index, columns=adj_weight.index)

# ret_forward = smpl.get_forward_return(data_replenish,'close')
# ret_forward.name = 'ret_forward'

# factors_filtered = factors_filtered.assign(industry = industy)
# # factors_filtered = factors_filtered.assign(country=1.0, industry = industy)
# factors_filtered = pd.get_dummies(factors_filtered,'industry')
# # del factors['country']

# factors_filtered

date        code  
2017-01-04  002032    1.000000
2017-01-05  002032    1.000000
2017-01-06  000792    0.231499
            002032    0.177793
            002074    0.205373
                        ...   
2021-12-30  603882    0.002785
            603899    0.003031
            603986    0.004244
            603993    0.004274
            605499    0.003310
Name: totalCapital, Length: 204548, dtype: float64

In [32]:
# adj_weight = np.sqrt(cap)/np.sqrt(cap).sum()
# adj_weight_mat = pd.DataFrame(np.diag(adj_weight),index=adj_weight.index, columns=adj_weight.index)

# factors_filtered
# excute_for_multidates(cap, lambda x:np.sqrt(x)/np.sqrt(x).sum())
# cap.loc[(slice(None),'000001')]

# from QUANTAXIS.QAData.data_marketvalue import QA_data_marketvalue
# QA_data_marketvalue
QA_data_marketvalue(data_replenish)

open    high     low   close    volume        amount  \
date       code                                                             
2017-01-04 000001    9.15    9.18    9.14    9.16  449329.0  4.115035e+08   
           000002   20.74   20.95   20.45   20.85  331554.0  6.867453e+08   
           000063   16.20   16.22   16.06   16.15  181859.0  2.933516e+08   
           000066   12.46   12.58   12.35   12.53  100596.0  1.254362e+08   
           000069    6.97    7.00    6.93    6.99  281692.0  1.962119e+08   
...                   ...     ...     ...     ...       ...           ...   
2021-12-30 603882  107.69  109.61  106.06  108.91   37180.0  4.026443e+08   
           603899   63.40   65.10   63.10   64.82   39949.0  2.578091e+08   
           603986  164.94  181.08  164.38  176.47  228363.0  3.987207e+09   
           603993    5.54    5.56    5.52    5.53  737165.0  4.084936e+08   
           605499  179.03  180.40  176.70  179.13    5823.0  1.041426e+08   

                         shares       lshares            mv  liquidity_mv  
date       code                                                            
2017-01-04 000001           NaN           NaN           NaN           NaN  
           000002           NaN           NaN           NaN           NaN  
           000063           NaN           NaN           NaN           NaN  
           000066           NaN           NaN           NaN           NaN  
           000069           NaN           NaN           NaN           NaN  
...                         ...           ...           ...           ...  
2021-12-30 603882  4.655863e+04  4.624626e+04  5.070700e+10  5.036680e+10  
           603899  9.270562e+04  9.220104e+04  6.009178e+10  5.976471e+10  
           603986  6.674355e+04  6.173563e+04  1.177824e+11  1.089449e+11  
           603993  2.159924e+06  1.766577e+06  1.194438e+11  9.769172e+10  
           605499  4.000100e+04  4.001000e+03  7.165379e+10  7.166991e+09  

[276813 rows x 10 columns]

In [7]:
# from QUANTAXIS.QAData.data_marketvalue import QA_data_marketvalue
test2 = smpl.get_data(['000001','000002','002032'],'2017-01-04','2017-02-28').data
# with pd.option_context("display.max_rows", None, "display.max_columns", None):
display(QA_data_marketvalue(test))

open   high    low  close    volume       amount  \
date       code                                                        
2017-01-04 000001   9.15   9.18   9.14   9.16  449329.0  411503456.0   
           000002  20.74  20.95  20.45  20.85  331554.0  686745344.0   
           002032  34.70  34.77  34.11  34.61    8743.0   30087142.0   
2017-01-05 000001   9.17   9.18   9.15   9.17  344372.0  315769696.0   
           000002  20.85  21.23  20.78  20.93  310125.0  651486080.0   
...                  ...    ...    ...    ...       ...          ...   
2017-02-27 000002  20.57  20.63  20.48  20.52  293446.0  602503168.0   
           002032  37.34  37.99  37.00  37.19    8059.0   30126162.0   
2017-02-28 000001   9.43   9.51   9.42   9.48  369719.0  350366880.0   
           000002  20.52  20.59  20.47  20.53  159762.0  327716640.0   
           002032  37.19  37.48  36.91  37.21   14893.0   55411064.0   

                         shares       lshares            mv  liquidity_mv  
date       code                                                            
2017-01-04 000001  1.430868e+06  1.219265e+06  1.310675e+11  1.116847e+11  
           000002  1.103915e+06  9.709068e+05  2.301663e+11  2.024341e+11  
           002032  6.317657e+04  4.194402e+04  2.186541e+10  1.451683e+10  
2017-01-05 000001  1.430868e+06  1.219265e+06  1.312106e+11  1.118066e+11  
           000002  1.103915e+06  9.709068e+05  2.310495e+11  2.032108e+11  
...                         ...           ...           ...           ...  
2017-02-27 000002  1.103915e+06  9.709068e+05  2.265234e+11  1.992301e+11  
           002032  6.317657e+04  4.194402e+04  2.349537e+10  1.559898e+10  
2017-02-28 000001  1.717041e+06  1.691799e+06  1.627755e+11  1.603825e+11  
           000002  1.103915e+06  9.709068e+05  2.266338e+11  1.993272e+11  
           002032  6.317657e+04  4.194402e+04  2.350800e+10  1.560737e+10  

[104 rows x 10 columns]